In [88]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import tensorflow as tf
from collections import Counter


%matplotlib inline

In [110]:
data1 = pd.read_csv("../Flawed/final_with_labels.csv")

In [115]:
mat = data1.as_matrix()

In [116]:
len(mat)

46785

In [114]:
Counter(mat[:,-8])

Counter({2004: 3225,
         2005: 3323,
         2006: 3462,
         2007: 3634,
         2008: 3729,
         2009: 3846,
         2010: 3913,
         2011: 4049,
         2012: 4155,
         2013: 4239,
         2014: 4484,
         2015: 4726})

In [108]:
#race = "white"
race = "hispanic"
#race = "mixed"

In [90]:
if race == "white":
    data = pd.read_csv("../Data/white/white_2year.csv")
if race == "hispanic":
    data = pd.read_csv("../Data/hispanic/hispanic_2year.csv")
if race == "mixed":
    data = pd.read_csv("../Data/mixed/mixed_2year.csv")
data_matrix = data.as_matrix()

In [91]:
data.head()

,pubcsnum,mar_stat,sex,primsite,histo3v,beho3v,grade,dx_conf,csexten,cslymphn,...,eod10_pn,eod10_ne,cstumsiz,maligcount,benbordcount,year_dx,nhiade,codpub,srv_time_mon,srv_time_mon_flag
0,7000085,5,1,C199,8210,3,2,1,160,0,...,98,0,999,4,0,2007,0,21040,47,1
1,7000085,9,1,C180,8140,3,2,1,600,300,...,97,98,40,4,0,2007,0,21040,47,1
2,7000321,5,2,C187,8210,3,1,1,200,0,...,0,2,8,2,0,2012,0,0,39,1
3,7000411,5,2,C187,8480,3,2,1,450,200,...,1,11,75,2,0,2006,0,21040,38,1
4,7000472,5,2,C199,8020,3,4,1,999,0,...,98,0,999,3,0,2009,0,21050,0,1


In [92]:
data_matrix.shape

(313122, 29)

In [93]:
data_matrix[0]

array([7000085, 5, 1, 'C199', 8210, 3, 2, 1, 160, 0, 0, 3.0, 0.0, 0.0, 26,
       0, 1, 1502, 77, 98, 0, 999, 4, 0, 2007, 0, 21040, 47, 1], dtype=object)

# Preprocessing

In [94]:
np.isin(data_matrix[:,-3], (21040, 21050))

array([ True,  True, False, ..., False,  True,  True], dtype=bool)

In [95]:
data_matrix[:,-2]

array([47, 47, 39, ..., 0, 35, 20], dtype=object)

In [96]:
# Y2 will be indicator of survival; 21040 and 21050 are SEER codes for colon and rectum cancer
Y2 = np.logical_not(np.logical_and(data_matrix[:,-2].astype(int)<24, np.isin(data_matrix[:,-3], (21040, 21050))))
X = data_matrix[:,1:23]

In [97]:
X[0]

array([5, 1, 'C199', 8210, 3, 2, 1, 160, 0, 0, 3.0, 0.0, 0.0, 26, 0, 1,
       1502, 77, 98, 0, 999, 4], dtype=object)

In [98]:
X_cat = X[:,:-5]
# primary site is only categorical variable with string labels
X_primsite = X[:,2]
X_cont = X[:,-5:]
# X_cat contains only categorical variables with integer labels
X_cat = np.delete(X_cat, [2], axis=1)

In [99]:
# Fill missing values in three eval columns with 9: unknown
# remember: index shifted over one since we deleted primsite
X_cat[:,9] = np.array([9 if np.isnan(x) else x for x in X_cat[:,9]])
X_cat[:,10] = np.array([9 if np.isnan(x) else x for x in X_cat[:,10]])
X_cat[:,11] = np.array([9 if np.isnan(x) else x for x in X_cat[:,11]])

In [100]:
# one-hot encode primary site
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
X_primsite_1hot = encoder.fit_transform(X_primsite)
encoder.classes_

array(['C180', 'C181', 'C182', 'C183', 'C184', 'C185', 'C186', 'C187',
       'C188', 'C189', 'C199', 'C209', 'C260'],
      dtype='<U4')

In [101]:
# one-hot encode remaining categorical variables
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
X_cat_1hot = encoder.fit_transform(X_cat)

In [102]:
# combine all categorical variables
X_cat = np.hstack((X_primsite_1hot, X_cat_1hot))

The continuous variables in order: age_dx, eod10_pn, eod10_ne, cstumsiz, maligcount. Age and maligcount do not require imputation, and eod10_ne (nodes examined) will not be used as a variable.


Here we will impute values for eod10_pn and cstumsiz. We will implement multivariate imputation by chained equations (MICE) on these two continuous variables. Their indices are and values which need to be imputed are:

1: eod10_pn (95, 97, 98, 99)<br>
3: cstumsiz (998, 999)<br>

Additionally, we will replace the values (991, 992, 993, 994, 995) in cstumsiz with (1, 2, 3, 4, 5) respectively.


### Note: The above should be re-verified for each different dataset. This is for colorectal cancer in Hispanic patients after 2004.

In [103]:
# implements MICE imputation with linear regression model on eod10_pn and cstumsiz
from sklearn.preprocessing import Imputer
from sklearn.linear_model import LinearRegression

NUM_CYCLES=10

missing_values_pn = (95, 97, 98, 99)
missing_values_tumsiz = (998, 999)

pn = np.array(X_cont[:,1]).reshape(-1,1)
tumsiz = np.array(X_cont[:,3]).reshape(-1,1)

# non-MICE imputation
for i, size in enumerate(tumsiz):
    if size not in (990, 991, 992, 993, 994, 995):
        continue
    elif size == 990:
        tumsiz[i] = 0
    elif size == 991:
        tumsiz[i] = 1
    elif size == 992:
        tumsiz[i] = 2
    elif size == 993:
        tumsiz[i] = 3
    elif size == 994:
        tumsiz[i] = 4
    elif size == 995:
        tumsiz[i] = 5


# set missing values to -1
for i, value in enumerate(pn):
    if value in missing_values_pn:
        pn[i] = -1
for i, value in enumerate(tumsiz):
    if value in missing_values_tumsiz:
        tumsiz[i] = -1

# indices for valid and imputation-needed positions in pn and tumsiz
idx_pn_missing = np.where(pn == -1)[0]
idx_pn_valid = np.where(pn != -1)[0]
idx_tumsiz_missing = np.where(tumsiz == -1)[0]
idx_tumsiz_valid = np.where(tumsiz != -1)[0]
    
# set missing values to mean
# copy = false modifies array in place
imp_pn = Imputer(missing_values = -1, copy=False)
imp_pn.fit_transform(pn)
imp_tumsiz = Imputer(missing_values = -1, copy=False)
imp_tumsiz.fit_transform(tumsiz)


for i in np.arange(NUM_CYCLES):
    print("Cycle " + str(i+1))
    
    # fit pn
    X_ind = np.hstack((X_cat[idx_pn_valid], tumsiz[idx_pn_valid]))
    X_rep = np.hstack((X_cat[idx_pn_missing], tumsiz[idx_pn_missing]))
    lin_reg = LinearRegression()
    lin_reg.fit(X_ind, pn[idx_pn_valid])
    pn[idx_pn_missing] = lin_reg.predict(X_rep)
    pn = np.maximum(pn, 0)
    pn = np.minimum(pn, 100)
    
    # fit tumsiz
    X_ind = np.hstack((X_cat[idx_tumsiz_valid], pn[idx_tumsiz_valid]))
    X_rep = np.hstack((X_cat[idx_tumsiz_missing], pn[idx_tumsiz_missing]))
    lin_reg = LinearRegression()
    lin_reg.fit(X_ind, tumsiz[idx_tumsiz_valid])
    tumsiz[idx_tumsiz_missing] = lin_reg.predict(X_rep)
    tumsiz = np.maximum(tumsiz, 0)
    tumsiz = np.minimum(tumsiz, 990)


Cycle 1
Cycle 2
Cycle 3
Cycle 4
Cycle 5
Cycle 6
Cycle 7
Cycle 8
Cycle 9
Cycle 10


In [104]:
age_dx = np.array(X_cont[:,0])
malig = np.array(X_cont[:,4])
X = np.column_stack((X_cat, age_dx, pn, malig, tumsiz))
print(X.shape)

(313122, 374)


We save the preprocessed data to disk. They are now available for our models.

In [107]:
if race == "white":
    np.save('../Data/white/X.npy', X)
    np.save('../Data/white/Y2.npy', Y2)
if race == "hispanic":
    np.save('../Data/hispanic/X.npy', X)
    np.save('../Data/hispanic/Y2.npy', Y2)
if race == "mixed":
    np.save('../Data/mixed/X.npy', X)
    np.save('../Data/mixed/Y2.npy', Y2)